# Common Agricultural Policy (CAP) Data 2014


In [ ]:
%matplotlib inline

from collections import OrderedDict
import json
import os

import pandas as pd

In [ ]:
DAERA = pd.read_excel('input/2014_All_CAP_Search_Results_Data_P14_1.xls', sheet_name=0)
SGRPID = pd.read_excel('input/2014_All_CAP_Search_Results_Data_P14_1.xls', sheet_name=1)
WG = pd.read_excel('input/2014_All_CAP_Search_Results_Data_P14_1.xls', sheet_name=2)
RPA = pd.read_excel('input/2014_All_CAP_Search_Results_Data_P14_1.xls', sheet_name=3)
RPA2 = pd.read_excel('input/2014_All_CAP_Search_Results_Data_P14_1.xls', sheet_name=4)

In [ ]:
RPA2.head()

In [ ]:
[RPA2.PayingAgencyLink.isna().sum(), RPA2.PayingAgencyLink.value_counts()]

It looks like we don't have quite the same set of columns, but it's just one junk column at the end:

In [ ]:
all_columns = set([
    column
    for df in [DAERA, SGRPID, WG, RPA, RPA2]
    for column in df.columns
])
[
    all_columns - set(DAERA.columns),
    all_columns - set(SGRPID.columns),
    all_columns - set(WG.columns),
    all_columns - set(RPA.columns),
    all_columns - set(RPA2.columns),
]

In [ ]:
[WG.Column1.unique(), RPA.Column1.unique(), RPA2.Column1.unique()]

In [ ]:
WG.drop('Column1', axis=1, inplace=True)
RPA.drop('Column1', axis=1, inplace=True)
RPA2.drop('Column1', axis=1, inplace=True)

In [ ]:
raw_cap = pd.concat([DAERA, SGRPID, WG, RPA, RPA2])
raw_cap.shape

In [ ]:
raw_cap.count()

## Postcode District Validation

Check the supplied postcode prefixes against a list of all valid postcode districts. 

In [ ]:
ukpostcodes = pd.read_csv('../postcodes/input/ukpostcodes.csv.gz')
ukpostcodes.shape

In [ ]:
ukpostcodes['district'] = ukpostcodes['postcode'].str.replace(r'^(.+)\s.+$', r'\1')
ukpostcodes['sector'] = ukpostcodes['postcode'].str.replace(r'^(.+)\s([0-9]).+$', r'\1 \2')

In [ ]:
ukpostcodes.head()

In [ ]:
postcode_districts = ukpostcodes['district'].unique()
len(postcode_districts)

In [ ]:
postcode_sectors = ukpostcodes['sector'].unique()
len(postcode_sectors)

In [ ]:
pd.merge(
    pd.DataFrame({'district': postcode_districts}),
    raw_cap,
    left_on='district', right_on='PostcodePrefix_F202B').shape

In [ ]:
raw_cap['postcode_district'] = raw_cap['PostcodePrefix_F202B'].str.upper().str.strip()
pd.merge(
    pd.DataFrame({'district': ukpostcodes['district'].unique()}),
    raw_cap,
    left_on='district', right_on='postcode_district').shape

In [ ]:
raw_cap[raw_cap['postcode_district'].isin(postcode_sectors)]

In [ ]:
def coarsen_sectors():
    is_sector = raw_cap['postcode_district'].isin(postcode_sectors)
    sectors = raw_cap['postcode_district'][is_sector]
    raw_cap.loc[is_sector, 'postcode_district'] = sectors.str.replace(r'^(.+)\s[0-9]$', r'\1')
coarsen_sectors()
pd.merge(
    pd.DataFrame({'district': postcode_districts}),
    raw_cap,
    left_on='district', right_on='postcode_district').shape

In [ ]:
def find_unmatched_districts():
    unmatched = raw_cap[~raw_cap['postcode_district'].isin(postcode_districts)]
    pairs = unmatched[['PostcodePrefix_F202B', 'TownCity_F202C']]
    return pd.DataFrame({
        'unmatched': pairs.apply(
            lambda x: str(x.PostcodePrefix_F202B) + ' / ' + str(x.TownCity_F202C), axis=1).unique()
    }).sort_values('unmatched')
find_unmatched_districts()

most of these we shouldn't expect to match

In [ ]:
cap = raw_cap[raw_cap['postcode_district'].isin(postcode_districts)].copy()
cap.shape

In [ ]:
cap[[
    'Year', 'BeneficiaryCode', 'BeneficiaryName_F201',
    'OtherEAGFTotal', 'DirectEAGFTotal', 'RuralDevelopmentTotal',
    'postcode_district']].to_pickle('output/cap_2014.pkl.gz')

## Aggregation to Postcode District

In [ ]:
cap_by_district = cap.groupby(['PayingAgencyLink', 'postcode_district']).aggregate(OrderedDict([
    ('OtherEAGFTotal', sum),
    ('DirectEAGFTotal', sum),
    ('RuralDevelopmentTotal', sum),
    ('Total', [sum, len]),
    ('Year', max)
]))
cap_by_district.reset_index(inplace=True)
cap_by_district.columns = [
    'agency',
    'postcode_district',
    'otherEAGF',
    'directEAGF',
    'ruralDevelopment',
    'total',
    'count',
    'year'
]
PROPERTY_COLUMNS = [
    'otherEAGF', 'directEAGF', 'ruralDevelopment', 'total', 'count'
]
for column in PROPERTY_COLUMNS:
    cap_by_district[column] = cap_by_district[column].round().astype('int32')
    
cap_by_district.shape

In [ ]:
cap_by_district.head()

In [ ]:
cap_by_district.agency.unique()

In [ ]:
cap_by_district.describe()

## Aggregation to Postcode Area

In [ ]:
cap['postcode_area'] = \
    cap['postcode_district'].str.replace(r'^([A-Z]{1,2}).+$', r'\1')
cap.head()

In [ ]:
cap_by_area = cap.groupby('postcode_area').sum()

cap_by_area = cap.groupby('postcode_area').aggregate(OrderedDict([
    ('OtherEAGFTotal', sum),
    ('DirectEAGFTotal', sum),
    ('RuralDevelopmentTotal', sum),
    ('Total', [sum, len]),
    ('Year', max)
]))
cap_by_area.reset_index(inplace=True)
cap_by_area.columns = [
    'postcode_area',
    'otherEAGF',
    'directEAGF',
    'ruralDevelopment',
    'total',
    'count',
    'year'
]
print(cap_by_area['total'].max()) # still a 32-bit integer?
for column in PROPERTY_COLUMNS:
    cap_by_area[column] = cap_by_area[column].round().astype('int32')
cap_by_area.head()

In [ ]:
cap_by_area.describe()

In [ ]:
cap_by_area.to_pickle('output/cap_by_area_2014.pkl.gz')